In [26]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy
import time
from typing import Union
import collections

out_count_observable_error_combos: Union[None, collections.Counter]  # 移除 [str]

import bposd.hgp

from bposd.hgp import hgp
import multiprocessing as mp
from ldpc.codes import ring_code
import stim
import re
import sys
sys.path.append("./src/")
import pymatching
import sinter
from typing import *
import re
def QECCircuit_OneStage(H_Z,H_X,J_X_1,J_X,J_Z,I_s,hz,hx,lx,lz, num_rep, circuit_error_params, p,circuit_type,D):
        
        #H_Z_deformed,H_X_deformed,hz,hx,lx=deformed_code_surface_code(hz,hx,lx,D)
        #H_Z_deformed,H_X_deformed,JX1JX1,JX2JX2,J_X,hz,hx,J_ZC1,J_ZC = deformed_code_magic(H_Z,H_X,J_X_1,J_X,J_Z,I_s,hz,hx,lx,lz,D) 
        #print(H_Z_deformed@J_X.T)
        J_ZC1 = block_diagonal(J_Z)#,lz,lz)
        J_ZC = J_ZC1
        H_X_deformed = block_diagonal(H_X)#,hx,hx)
        H_Z_deformed = block_diagonal(H_Z)#,hz,hz)
        print(H_X_deformed.shape)
        hx = H_X_deformed
        hz = H_Z_deformed
        print("wohaoshuai")
        M = hx@hz.T%2
        print(remove_zero_rows(M))
        print(remove_zero_rows(H_X_deformed@J_ZC1.T%2))
        if circuit_type == 'coloration':
           scheduling_X = ColorationCircuit(hx)
           scheduling_Z = ColorationCircuit(hz)

        H_X = hx
        H_Z = hz
        data_indices = list(np.arange(0, np.shape(H_X)[1]))
        data_indicesancilla = list(np.arange(np.shape(H_X)[1], np.shape(H_X_deformed)[1]))
        n = len(data_indices) + len(data_indicesancilla)
        n_Z_ancilla, n_X_ancilla = np.shape(H_Z)[0], np.shape(H_X)[0]
        n_Z_ancillaM, n_X_ancillaM =  np.shape(H_Z_deformed)[0]- np.shape(H_Z)[0], np.shape(H_X_deformed)[0]- np.shape(H_X)[0]

        Z_ancilla_indices = list(np.arange(n, n + n_Z_ancilla))
        Z_ancilla_indicesM = list(np.arange(n + n_Z_ancilla ,
                                        n + n_Z_ancilla  + n_Z_ancillaM))
        X_ancilla_indices = list(np.arange(n + n_Z_ancilla+ n_Z_ancillaM, n + n_Z_ancilla+ n_Z_ancillaM + n_X_ancilla))
        X_ancilla_indicesM = list(np.arange(n + n_Z_ancilla + n_Z_ancillaM + n_X_ancilla,
                                        n + n_Z_ancilla+ n_Z_ancillaM + n_X_ancilla + n_X_ancillaM))
        print(n)
        print(n_Z_ancilla)
        print(n_Z_ancillaM)
        print(n_X_ancilla )
        print(n_X_ancillaM)
        circuit_init = stim.Circuit()
        circuit_init.append("R", data_indices)
        circuit_init.append("R", X_ancilla_indices )
        circuit_init.append("R",  Z_ancilla_indices)
        circuit_init.append("R", data_indicesancilla)
        
        circuit_init.append("R", X_ancilla_indicesM )
        circuit_init.append("R",  Z_ancilla_indicesM)
        circuit_stab_meas = stim.Circuit()
        circuit_stab_meas.append("H", X_ancilla_indices)
        
        circuit_stab_meas.append("TICK")
        #circuit_stab_meas.append("DEPOLARIZE1", data_indices  , (0.01))\debug
        for time_step in range(len(scheduling_X)):
            for j in scheduling_X[time_step]:
#                 supported_data_qubits = list(np.where(hx[X_ancilla_index - n - n_Z_ancilla,:] == 1)[0])
                X_ancilla_index = X_ancilla_indices[j]
                data_index = scheduling_X[time_step][j]
                circuit_stab_meas.append("CX", [X_ancilla_index, data_index])
            circuit_stab_meas.append("TICK")
        circuit_stab_meas.append("TICK")
        
        for time_step in range(len(scheduling_Z)):
            for j in scheduling_Z[time_step]:
#                 supported_data_qubits = list(np.where(hz[Z_ancilla_index - n,:] == 1)[0])
                Z_ancilla_index = Z_ancilla_indices[j]
                data_index = scheduling_Z[time_step][j]
                # data_index = supported_data_qubits[i]
                circuit_stab_meas.append("CX", [data_index, Z_ancilla_index])
            circuit_stab_meas.append("TICK")
        circuit_stab_meas.append("TICK")
        # Measure the ancillas
        # circuit_stab_meas.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas.append("H", X_ancilla_indices)
        circuit_stab_meas.append("MR", Z_ancilla_indices )
        circuit_stab_meas.append("MR", X_ancilla_indices)
        circuit_stab_meas.append("TICK")


        error_params = {"p_i": circuit_error_params["p_i"]*p, "p_state_p": circuit_error_params["p_state_p"]*p, 
        "p_m": circuit_error_params["p_m"]*p, "p_CX":circuit_error_params["p_CX"]*p, 
        "p_idling_gate": circuit_error_params["p_idling_gate"]*p,"p_H" : circuit_error_params["p_H"]*p,"p_id" : circuit_error_params["p_id"]*p} 
        print(error_params)

        circuit_stab_meas_Hz1 = stim.Circuit()


        circuit_stab_meas_Hz1.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz1.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_state_p']))
    #circuit_stab_meas_Hz1.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_H'])) # Add the state preparation error # Add the idling errors on the data qubits during the preparation for X ancillas
        circuit_stab_meas_Hz1.append("TICK")
        circuit_stab_meas_Hz1.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate'])) #+ X_ancilla_indices
    # Apply CX gates for the X stabilizers
        for time_step in range(len(scheduling_X)):
        # add idling errors for all the qubits during the ancilla shuffling
            idling_qubits = data_indices + X_ancilla_indices
            idling_data_indices = list(copy.deepcopy(data_indices))
        #circuit_stab_meas_Hz1.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate'])) 
            for j in scheduling_X[time_step]:
    #                 supported_data_qubits = list(np.where(hx[X_ancilla_index - n - n_Z_ancilla,:] == 1)[0])
                X_ancilla_index = X_ancilla_indices[j]
                data_index = scheduling_X[time_step][j]
            # data_index = supported_data_qubits[i]
                circuit_stab_meas_Hz1.append("CX", [X_ancilla_index, data_index])
                if data_index in idling_data_indices:
                     idling_data_indices.pop(idling_data_indices.index(data_index))
            circuit_stab_meas_Hz1.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
            circuit_stab_meas_Hz1.append("TICK")

    # meausure the Z ancillas
        circuit_stab_meas_Hz1.append("DEPOLARIZE1", Z_ancilla_indices, (error_params['p_state_p'])) # Add the state preparation error
    # circuit_stab_meas_Hz1.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the preparation for Z ancillas
    
        circuit_stab_meas_Hz1.append("DEPOLARIZE1",  data_indices , (error_params['p_idling_gate']))
        circuit_stab_meas_Hz1.append("TICK")
    # Appy CX gates for the Z stabilziers
        for time_step in range(len(scheduling_Z)):
              idling_qubits = data_indices + Z_ancilla_indices
              idling_data_indices = list(copy.deepcopy(data_indices))
        #circuit_stab_meas_Hz1.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
              for j in scheduling_Z[time_step]:
    #                 supported_data_qubits = list(np.where(hz[Z_ancilla_index - n,:] == 1)[0])
                  Z_ancilla_index = Z_ancilla_indices[j]
                  data_index = scheduling_Z[time_step][j]
            # data_index = supported_data_qubits[i]
                  circuit_stab_meas_Hz1.append("CX", [data_index, Z_ancilla_index])
                  if data_index in idling_data_indices:
                         idling_data_indices.pop(idling_data_indices.index(data_index))
              circuit_stab_meas_Hz1.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
              circuit_stab_meas_Hz1.append("TICK")

    # Measure the ancillas
        
        circuit_stab_meas_Hz1.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz1.append("TICK")
        circuit_stab_meas_Hz1.append("DEPOLARIZE1",  data_indices , (error_params['p_idling_gate']))
    #circuit_stab_meas_Hz1.append("DEPOLARIZE1",  X_ancilla_indices, (error_params['p_H']))
        circuit_stab_meas_Hz1.append("DEPOLARIZE1",  Z_ancilla_indices +X_ancilla_indices, (error_params['p_m'])) # Add the measurement error
        circuit_stab_meas_Hz1.append("TICK")
    # circuit_stab_meas_Hz1.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas_Hz1.append("MR",X_ancilla_indices)
        circuit_stab_meas_Hz1.append("MR", Z_ancilla_indices )
    #circuit_stab_meas_Hz1.append("DEPOLARIZE1", X_ancilla_indices+Z_ancilla_indices, (error_params['p_i']))
        circuit_stab_meas_Hz1.append("SHIFT_COORDS", [], (1))
        for i in range(len(Z_ancilla_indices)):
                 circuit_stab_meas_Hz1.append("DETECTOR", [stim.target_rec(-len(Z_ancilla_indices)+ i)], (0))
        circuit_stab_meas_Hz1.append("TICK")

    # rep with difference detectors
        circuit_stab_meas_Hz2 = stim.Circuit()
    
    # measurement the X ancillas
    # # Initialize the X ancillas to the + state

        circuit_stab_meas_Hz2.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz2.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_state_p'])) 
    #circuit_stab_meas_Hz2.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_H'])) # Add the state preparation error
    #ircuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the preparation for X ancillas
        circuit_stab_meas_Hz2.append("TICK")
        circuit_stab_meas_Hz2.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate']))
    # Apply CX gates for the X stabilizers
        for time_step in range(len(scheduling_X)):
                     idling_qubits = data_indices + X_ancilla_indices
        #circuit_stab_meas_Hz2.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
                     idling_data_indices = list(copy.deepcopy(data_indices))
                     for j in scheduling_X[time_step]:
    #                 supported_data_qubits = list(np.where(hx[X_ancilla_index - n - n_Z_ancilla,:] == 1)[0])
                           X_ancilla_index = X_ancilla_indices[j]
                           data_index = scheduling_X[time_step][j]
            # data_index = supported_data_qubits[i]
                           circuit_stab_meas_Hz2.append("CX", [X_ancilla_index, data_index])
                           if data_index in idling_data_indices:
                                  idling_data_indices.pop(idling_data_indices.index(data_index))
                     circuit_stab_meas_Hz2.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
                     circuit_stab_meas_Hz2.append("TICK")

    # meausure the Z ancillas
    ## initialize the Z ancillas
        circuit_stab_meas_Hz2.append("DEPOLARIZE1", Z_ancilla_indices, (error_params['p_state_p'])) # Add the state preparation error
        circuit_stab_meas_Hz2.append("DEPOLARIZE1", data_indices, (error_params['p_idling_gate'])) # Add the idling errors on the data qubits during the preparation for Z ancillas
        circuit_stab_meas_Hz2.append("TICK")
    # Appy CX gates for the Z stabilziers
    #circuit_stab_meas_Hz2.append("DEPOLARIZE1",  data_indices + Z_ancilla_indices, (error_params['p_idling_gate']))
        for time_step in range(len(scheduling_Z)):
                  idling_qubits = data_indices + Z_ancilla_indices
        #circuit_stab_meas_Hz2.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
                  idling_data_indices = list(copy.deepcopy(data_indices))
                  for j in scheduling_Z[time_step]:
    #                 supported_data_qubits = list(np.where(hz[Z_ancilla_index - n,:] == 1)[0])
                       Z_ancilla_index = Z_ancilla_indices[j]
                       data_index = scheduling_Z[time_step][j]
            # data_index = supported_data_qubits[i]
                       circuit_stab_meas_Hz2.append("CX", [data_index, Z_ancilla_index])
                       if data_index in idling_data_indices:
                                 idling_data_indices.pop(idling_data_indices.index(data_index))
                  circuit_stab_meas_Hz2.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
                  circuit_stab_meas_Hz2.append("TICK")        

    # Measure the ancillas
        circuit_stab_meas_Hz2.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz2.append("TICK")
        circuit_stab_meas_Hz2.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate'])) 
    #circuit_stab_meas_Hz2.append("DEPOLARIZE1",  X_ancilla_indices, (error_params['p_H']))
        circuit_stab_meas_Hz2.append("DEPOLARIZE1",  Z_ancilla_indices +X_ancilla_indices, (error_params['p_m']))# Add the measurement error
        circuit_stab_meas_Hz2.append("TICK")
    # circuit_stab_meas_Hz2.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas_Hz2.append("MR",  X_ancilla_indices)
        circuit_stab_meas_Hz2.append("MR",  Z_ancilla_indices)
    #circuit_stab_meas_Hz2.append("SHIFT_COORDS", [], (1))
        for i in range(len(Z_ancilla_indices)):
            circuit_stab_meas_Hz2.append("DETECTOR", [stim.target_rec(-len(Z_ancilla_indices) + i), 
                                        stim.target_rec(-len(Z_ancilla_indices)+ i - len(Z_ancilla_indices) - len(X_ancilla_indices))], (0))
        circuit_stab_meas_Hz2.append("TICK")


        
        if circuit_type == 'coloration':
            scheduling_X_deform = ColorationCircuit(H_X_deformed)
            scheduling_Z_deform = ColorationCircuit(H_Z_deformed)
        circuit_stab_meas_rep1 = stim.Circuit()
        circuit_stab_meas_rep1.append("DEPOLARIZE1", data_indicesancilla, (error_params['p_state_p']))
        circuit_stab_meas_rep1.append("TICK")
        circuit_stab_meas_rep1.append("H", X_ancilla_indices+X_ancilla_indicesM)
        circuit_stab_meas_rep1.append("DEPOLARIZE1", X_ancilla_indices+X_ancilla_indicesM, (error_params['p_state_p']))
        
        circuit_stab_meas_rep1.append("TICK") # Add the state preparation error
        circuit_stab_meas_rep1.append("DEPOLARIZE1", data_indices, (error_params['p_id'])) # Add the idling errors on the data qubits during the preparation for X ancillas
        circuit_stab_meas_rep1.append("TICK")
        # Apply CX gates for the X stabilizers
        #circuit_stab_meas_rep1.append("DEPOLARIZE1", data_indices + data_indicesancilla , (error_params['p_idling_gate']))#+ X_ancilla_indices+X_ancilla_indicesM
        for time_step in range(len(scheduling_X_deform)):
            # add idling errors for all the qubits during the ancilla shuffling
            idling_qubits = data_indices + data_indicesancilla + X_ancilla_indices+X_ancilla_indicesM
            idling_data_indices = list(copy.deepcopy(data_indices+data_indicesancilla))
            #circuit_stab_meas_rep1.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate'])) ##########################
            for j in scheduling_X_deform[time_step]:
#               
                X_ancilla_index = (X_ancilla_indices+X_ancilla_indicesM)[j]
                data_index = scheduling_X_deform[time_step][j]
                
                circuit_stab_meas_rep1.append("CX", [X_ancilla_index, data_index])
                if data_index in idling_data_indices:
                    idling_data_indices.pop(idling_data_indices.index(data_index))
            circuit_stab_meas_rep1.append("DEPOLARIZE1", idling_data_indices, (error_params['p_id'])) # idling errors for qubits that are not being checked
            circuit_stab_meas_rep1.append("TICK")
        circuit_stab_meas_rep1.append("DEPOLARIZE1", data_indices+ data_indicesancilla , (error_params['p_id']))#idling errors for qubits that are not being Measurement
        circuit_stab_meas_rep1.append("DEPOLARIZE1", Z_ancilla_indices+Z_ancilla_indicesM, (error_params['p_state_p'])) # Add the state preparation error
        circuit_stab_meas_rep1.append("TICK")
        #circuit_stab_meas_rep1.append("DEPOLARIZE1", data_indices+ data_indicesancilla + Z_ancilla_indices+Z_ancilla_indicesM, (error_params['p_id']))
        for time_step in range(len(scheduling_Z_deform)):
            idling_qubits = data_indices+ data_indicesancilla + Z_ancilla_indices+Z_ancilla_indicesM
            idling_data_indices = list(copy.deepcopy(data_indices+data_indicesancilla))
            #circuit_stab_meas_rep1.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
            for j in scheduling_Z_deform[time_step]:
#                 
                Z_ancilla_index = (Z_ancilla_indices+Z_ancilla_indicesM)[j]
                data_index = scheduling_Z_deform[time_step][j]
                
                circuit_stab_meas_rep1.append("CX", [data_index, Z_ancilla_index])
                if data_index in idling_data_indices:
                    idling_data_indices.pop(idling_data_indices.index(data_index))
            circuit_stab_meas_rep1.append("DEPOLARIZE1", idling_data_indices, (error_params['p_id'])) # idling errors for qubits that are not being checked
            circuit_stab_meas_rep1.append("TICK")
        circuit_stab_meas_rep1.append("DEPOLARIZE1", data_indices+data_indicesancilla, (error_params['p_id']))
        # Measure the ancillas
        circuit_stab_meas_rep1.append("H", X_ancilla_indices+ X_ancilla_indicesM)
        circuit_stab_meas_rep1.append("TICK")
        circuit_stab_meas_rep1.append("DEPOLARIZE1", X_ancilla_indices+ X_ancilla_indicesM+Z_ancilla_indices+Z_ancilla_indicesM , (error_params['p_m']))

        circuit_stab_meas_rep1.append("TICK")
        #circuit_stab_meas_rep1.append("DEPOLARIZE1",  X_ancilla_indices+ X_ancilla_indicesM, (3/2*error_params['p_m'])) # Add the measurement error
        #circuit_final_mea.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas_rep1.append("MR", Z_ancilla_indices+Z_ancilla_indicesM )
        circuit_stab_meas_rep1.append("MR", X_ancilla_indices+X_ancilla_indicesM)
        #circuit_stab_meas_rep1.append("RX", X_ancilla_indices+X_ancilla_indicesM)
        circuit_stab_meas_rep1.append("TICK")
        circuit_stab_meas_rep1.append("SHIFT_COORDS", [], (1))
        for i in range(len(Z_ancilla_indices)):
            circuit_stab_meas_rep1.append("DETECTOR", [stim.target_rec(- len(Z_ancilla_indices)-len(Z_ancilla_indicesM)- len(X_ancilla_indices)-len(X_ancilla_indicesM) + i)
                                                      ,stim.target_rec(- len(Z_ancilla_indices)- len(X_ancilla_indices)- len(Z_ancilla_indices)-len(Z_ancilla_indicesM)- len(X_ancilla_indices)-len(X_ancilla_indicesM) + i)], (0))
        for i in range(len(Z_ancilla_indicesM)):
            circuit_stab_meas_rep1.append("DETECTOR", [stim.target_rec(-len(Z_ancilla_indicesM)- len(X_ancilla_indices)-len(X_ancilla_indicesM) + i)], (0))
  
        circuit_stab_meas_rep1.append("TICK")
        # rep with difference detectors
        circuit_stab_meas_rep2 = stim.Circuit()
        #circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices+data_indicesancilla , (error_params['p_m']))
        # measurement the X ancillas
        # # Initialize the X ancillas to the + state
#         circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices, (pz)) # for debug
        #circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices + data_indicesancilla, (error_params['p_id']))
        circuit_stab_meas_rep2.append("DEPOLARIZE1", X_ancilla_indices+X_ancilla_indicesM, (error_params['p_state_p']))
        circuit_stab_meas_rep2.append("TICK")# Add the state preparation error
        circuit_stab_meas_rep2.append("H", X_ancilla_indices+X_ancilla_indicesM)
        circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices + data_indicesancilla, (error_params['p_id'])) 
        # circuit_final_mea.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the preparation for X ancillas
        circuit_stab_meas_rep2.append("TICK")
        #circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices + data_indicesancilla + X_ancilla_indices+X_ancilla_indicesM, (error_params['p_idling_gate']))
        # Apply CX gates for the X stabilizers
        for time_step in range(len(scheduling_X_deform)):
            # add idling errors for all the qubits during the ancilla shuffling
            idling_qubits = data_indices + data_indicesancilla + X_ancilla_indices+X_ancilla_indicesM
            idling_data_indices = list(copy.deepcopy(data_indices+data_indicesancilla))
            #circuit_stab_meas_rep2.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate'])) 
            for j in scheduling_X_deform[time_step]:
#                 supported_data_qubits = list(np.where(hx[X_ancilla_index - n - n_Z_ancilla,:] == 1)[0])
                X_ancilla_index = (X_ancilla_indices+X_ancilla_indicesM)[j]
                data_index = scheduling_X_deform[time_step][j]
                # data_index = supported_data_qubits[i]
                circuit_stab_meas_rep2.append("CX", [X_ancilla_index, data_index])
                if data_index in idling_data_indices:
                    idling_data_indices.pop(idling_data_indices.index(data_index))
            circuit_stab_meas_rep2.append("DEPOLARIZE1", idling_data_indices, (error_params['p_id'])) # idling errors for qubits that are not being checked
            circuit_stab_meas_rep2.append("TICK")
        circuit_stab_meas_rep2.append("DEPOLARIZE1", Z_ancilla_indices+Z_ancilla_indicesM, (error_params['p_state_p'])) # Add the state preparation error
        circuit_stab_meas_rep2.append("TICK")
        circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices+ data_indicesancilla , (error_params['p_id']))
        for time_step in range(len(scheduling_Z_deform)):
            idling_qubits = data_indices+ data_indicesancilla + Z_ancilla_indices+Z_ancilla_indicesM
            idling_data_indices = list(copy.deepcopy(data_indices+data_indicesancilla))
            #circuit_stab_meas_rep2.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
            for j in scheduling_Z_deform[time_step]:
#                 supported_data_qubits = list(np.where(hz[Z_ancilla_index - n,:] == 1)[0])
                Z_ancilla_index = (Z_ancilla_indices+Z_ancilla_indicesM)[j]
                data_index = scheduling_Z_deform[time_step][j]
                # data_index = supported_data_qubits[i]
                circuit_stab_meas_rep2.append("CX", [data_index, Z_ancilla_index])
                if data_index in idling_data_indices:
                    idling_data_indices.pop(idling_data_indices.index(data_index))
            circuit_stab_meas_rep2.append("DEPOLARIZE1", idling_data_indices, (error_params['p_id'])) # idling errors for qubits that are not being checked
            circuit_stab_meas_rep2.append("TICK")
        circuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices+ data_indicesancilla , (error_params['p_id']))
        # Measure the ancillas
        circuit_stab_meas_rep2.append("TICK")
        circuit_stab_meas_rep2.append("H", X_ancilla_indices+ X_ancilla_indicesM)
        circuit_stab_meas_rep2.append("DEPOLARIZE1", Z_ancilla_indices+Z_ancilla_indicesM+X_ancilla_indices+ X_ancilla_indicesM, (error_params['p_m'])) # Add the measurement error
        # circuit_final_mea.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas_rep2.append("TICK")
        circuit_stab_meas_rep2.append("MR", Z_ancilla_indices+Z_ancilla_indicesM )
        circuit_stab_meas_rep2.append("MR", X_ancilla_indices+X_ancilla_indicesM)
        #circuit_stab_meas_rep1.append("RX", X_ancilla_indices+X_ancilla_indicesM)
        circuit_stab_meas_rep2.append("SHIFT_COORDS", [], (1))            
        for i in range(len(Z_ancilla_indices)+len(Z_ancilla_indicesM)):
            circuit_stab_meas_rep2.append("DETECTOR", [stim.target_rec(- len(X_ancilla_indices)-len(X_ancilla_indicesM)- len(Z_ancilla_indices)-len(Z_ancilla_indicesM) + i), 
                                            stim.target_rec( -len(Z_ancilla_indices)-len(Z_ancilla_indicesM) - len(X_ancilla_indices)-len(X_ancilla_indicesM) + i - len(Z_ancilla_indices) -len(Z_ancilla_indicesM)- len(X_ancilla_indices)-len(X_ancilla_indicesM))], (0))
        circuit_stab_meas_rep2.append("TICK")
        circuit_stab_meas_Hz3 = stim.Circuit()
        circuit_stab_meas_Hz3.append("DEPOLARIZE1",  data_indicesancilla, (error_params['p_m']))
        circuit_stab_meas_Hz3.append("MR", data_indicesancilla)   
        circuit_stab_meas_Hz3.append("TICK")
    # measurement the X ancillas
    # # Initialize the X ancillas to the + state
        circuit_stab_meas_Hz3.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz3.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_state_p'])) 


        

    #circuit_stab_meas_Hz3.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_H'])) # Add the state preparation error
    #ircuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the preparation for X ancillas
        circuit_stab_meas_Hz3.append("TICK")
        circuit_stab_meas_Hz3.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate']))
    # Apply CX gates for the X stabilizers
        for time_step in range(len(scheduling_X)):
                     idling_qubits = data_indices + X_ancilla_indices
        #circuit_stab_meas_Hz3.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
                     idling_data_indices = list(copy.deepcopy(data_indices))
                     for j in scheduling_X[time_step]:
    #                 supported_data_qubits = list(np.where(hx[X_ancilla_index - n - n_Z_ancilla,:] == 1)[0])
                           X_ancilla_index = X_ancilla_indices[j]
                           data_index = scheduling_X[time_step][j]
            # data_index = supported_data_qubits[i]
                           circuit_stab_meas_Hz3.append("CX", [X_ancilla_index, data_index])
                           if data_index in idling_data_indices:
                                  idling_data_indices.pop(idling_data_indices.index(data_index))
                     circuit_stab_meas_Hz3.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
                     circuit_stab_meas_Hz3.append("TICK")

    # meausure the Z ancillas
    ## initialize the Z ancillas
        circuit_stab_meas_Hz3.append("DEPOLARIZE1", Z_ancilla_indices, (error_params['p_state_p'])) # Add the state preparation error
        circuit_stab_meas_Hz3.append("DEPOLARIZE1", data_indices, (error_params['p_idling_gate'])) # Add the idling errors on the data qubits during the preparation for Z ancillas
        circuit_stab_meas_Hz3.append("TICK")
    # Appy CX gates for the Z stabilziers
    #circuit_stab_meas_Hz3.append("DEPOLARIZE1",  data_indices + Z_ancilla_indices, (error_params['p_idling_gate']))
        for time_step in range(len(scheduling_Z)):
                  idling_qubits = data_indices + Z_ancilla_indices
        #circuit_stab_meas_Hz3.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
                  idling_data_indices = list(copy.deepcopy(data_indices))
                  for j in scheduling_Z[time_step]:
    #                 supported_data_qubits = list(np.where(hz[Z_ancilla_index - n,:] == 1)[0])
                       Z_ancilla_index = Z_ancilla_indices[j]
                       data_index = scheduling_Z[time_step][j]
            # data_index = supported_data_qubits[i]
                       circuit_stab_meas_Hz3.append("CX", [data_index, Z_ancilla_index])
                       if data_index in idling_data_indices:
                                 idling_data_indices.pop(idling_data_indices.index(data_index))
                  circuit_stab_meas_Hz3.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
                  circuit_stab_meas_Hz3.append("TICK")
         
        circuit_stab_meas_Hz3.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate']))
    # Measure the ancillas
        circuit_stab_meas_Hz3.append("H", X_ancilla_indices)
    #circuit_stab_meas_Hz3.append("DEPOLARIZE1",  X_ancilla_indices, (error_params['p_H']))
        circuit_stab_meas_Hz3.append("DEPOLARIZE1",  Z_ancilla_indices +X_ancilla_indices, (error_params['p_m']))# Add the measurement error
        circuit_stab_meas_Hz3.append("TICK")
    # circuit_stab_meas_Hz3.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas_Hz3.append("MR", Z_ancilla_indices + X_ancilla_indices)
    #circuit_stab_meas_Hz3.append("SHIFT_COORDS", [], (1))
        for i in range(len(Z_ancilla_indices)):
            supported_data_indices = list(np.where(H_Z_deformed[i,len(data_indices):] == 1)[0])#######
            rec_indices = []
            for data_index in supported_data_indices:
                rec_indices.append(-len(data_indicesancilla)-len(Z_ancilla_indices)- len(X_ancilla_indices) + data_index)
            rec_indices.append(-len(Z_ancilla_indices)- len(X_ancilla_indices) + i )
            circuit_stab_meas_Hz3.append("Detector", [stim.target_rec(rec_index) for rec_index in rec_indices], (0))

    
        for i in range(len(Z_ancilla_indicesM)):
            supported_data_indices = list(np.where(H_Z_deformed[i+len(Z_ancilla_indices),:] == 1)[0])#######
            rec_indices = []
            for data_index in supported_data_indices:
                rec_indices.append(-len(data_indices)-len(data_indicesancilla)-len(Z_ancilla_indices)- len(X_ancilla_indices) + data_index)
            circuit_stab_meas_Hz3.append("Detector", [stim.target_rec(rec_index) for rec_index in rec_indices], (0))                                                
            circuit_stab_meas_Hz3.append("TICK")
        circuit_stab_meas_Hz4 = stim.Circuit()
    
    # measurement the X ancillas
    # # Initialize the X ancillas to the + state

        circuit_stab_meas_Hz4.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz4.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_state_p'])) 
    #circuit_stab_meas_Hz4.append("DEPOLARIZE1", X_ancilla_indices, (error_params['p_H'])) # Add the state preparation error
    #ircuit_stab_meas_rep2.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the preparation for X ancillas
        circuit_stab_meas_Hz4.append("TICK")
        circuit_stab_meas_Hz4.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate']))
    # Apply CX gates for the X stabilizers
        for time_step in range(len(scheduling_X)):
                     idling_qubits = data_indices + X_ancilla_indices
        #circuit_stab_meas_Hz4.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
                     idling_data_indices = list(copy.deepcopy(data_indices))
                     for j in scheduling_X[time_step]:
    #                 supported_data_qubits = list(np.where(hx[X_ancilla_index - n - n_Z_ancilla,:] == 1)[0])
                           X_ancilla_index = X_ancilla_indices[j]
                           data_index = scheduling_X[time_step][j]
            # data_index = supported_data_qubits[i]
                           circuit_stab_meas_Hz4.append("CX", [X_ancilla_index, data_index])
                           if data_index in idling_data_indices:
                                  idling_data_indices.pop(idling_data_indices.index(data_index))
                     circuit_stab_meas_Hz4.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
                     circuit_stab_meas_Hz4.append("TICK")

    # meausure the Z ancillas
    ## initialize the Z ancillas
        circuit_stab_meas_Hz4.append("DEPOLARIZE1", Z_ancilla_indices, (error_params['p_state_p'])) # Add the state preparation error
        circuit_stab_meas_Hz4.append("DEPOLARIZE1", data_indices, (error_params['p_idling_gate'])) # Add the idling errors on the data qubits during the preparation for Z ancillas
        circuit_stab_meas_Hz4.append("TICK")
    # Appy CX gates for the Z stabilziers
    #circuit_stab_meas_Hz4.append("DEPOLARIZE1",  data_indices + Z_ancilla_indices, (error_params['p_idling_gate']))
        for time_step in range(len(scheduling_Z)):
                  idling_qubits = data_indices + Z_ancilla_indices
        #circuit_stab_meas_Hz4.append("DEPOLARIZE1", idling_qubits, (error_params['p_idling_gate']))
                  idling_data_indices = list(copy.deepcopy(data_indices))
                  for j in scheduling_Z[time_step]:
    #                 supported_data_qubits = list(np.where(hz[Z_ancilla_index - n,:] == 1)[0])
                       Z_ancilla_index = Z_ancilla_indices[j]
                       data_index = scheduling_Z[time_step][j]
            # data_index = supported_data_qubits[i]
                       circuit_stab_meas_Hz4.append("CX", [data_index, Z_ancilla_index])
                       if data_index in idling_data_indices:
                                 idling_data_indices.pop(idling_data_indices.index(data_index))
                  circuit_stab_meas_Hz4.append("DEPOLARIZE1", idling_data_indices, (error_params['p_idling_gate'])) # idling errors for qubits that are not being checked
                  circuit_stab_meas_Hz4.append("TICK")        

    # Measure the ancillas
        circuit_stab_meas_Hz4.append("H", X_ancilla_indices)
        circuit_stab_meas_Hz4.append("TICK")
        circuit_stab_meas_Hz4.append("DEPOLARIZE1", data_indices , (error_params['p_idling_gate'])) 
    #circuit_stab_meas_Hz4.append("DEPOLARIZE1",  X_ancilla_indices, (error_params['p_H']))
        circuit_stab_meas_Hz4.append("DEPOLARIZE1",  Z_ancilla_indices +X_ancilla_indices, (error_params['p_m']))# Add the measurement error
        circuit_stab_meas_Hz4.append("TICK")
    # circuit_stab_meas_Hz4.append("DEPOLARIZE1", data_indices, (error_params['p_i'])) # Add the idling errors on the data qubits during the measurement of X ancillas
        circuit_stab_meas_Hz4.append("MR", Z_ancilla_indices + X_ancilla_indices)
    #circuit_stab_meas_Hz4.append("SHIFT_COORDS", [], (1))
        for i in range(len(Z_ancilla_indices)):
            circuit_stab_meas_Hz4.append("DETECTOR", [stim.target_rec(-len(Z_ancilla_indices)- len(X_ancilla_indices) + i), 
                                        stim.target_rec(-len(Z_ancilla_indices)- len(X_ancilla_indices) + i - len(Z_ancilla_indices) - len(X_ancilla_indices))], (0))
        circuit_stab_meas_Hz4.append("TICK")




##########################################################
    # final transversal readout
        circuit_final_meas = stim.Circuit()
    #         circuit_final_meas_f.append("DEPOLARIZE1", data_indices, (1*pz)) # for debug
        circuit_final_meas.append("DEPOLARIZE1",  data_indices, (error_params['p_m'])) # Add the measurement error
        circuit_final_meas.append("MR", data_indices)
        circuit_final_meas.append("SHIFT_COORDS", [], (1))
    # Obtain the syndroms
        for i in range(len(Z_ancilla_indices)):
           supported_data_indices = list(np.where(H_Z[i,:] == 1)[0])
           rec_indices = []
           for data_index in supported_data_indices:
               rec_indices.append(- len(data_indices) + data_index)
           rec_indices.append(-len(Z_ancilla_indices) + i - len(data_indices))######################################
           circuit_final_meas.append("Detector", [stim.target_rec(rec_index) for rec_index in rec_indices], (0))
    # Obtain the logical measurements result
        for i in range(len(J_ZC)):
           logical_X_qubit_indices = list(np.where(J_ZC[i,:] == 1)[0])
           circuit_final_meas.append("OBSERVABLE_INCLUDE", 
                           [stim.target_rec(- len(data_indices) + data_index) for data_index in logical_X_qubit_indices],
                           (i))
        circuit_stab_meas_rep = circuit_stab_meas_Hz1+ (num_rep-1)*circuit_stab_meas_Hz2
        circuit_stab_meas_rep = AddCXError(circuit_stab_meas_rep, 'DEPOLARIZE2(%f)' % error_params["p_CX"])
        noisy_circuit =circuit_init+circuit_stab_meas+circuit_stab_meas_rep+circuit_final_meas

    
        return noisy_circuit

    
          
def AddCXError(circuit:stim.Circuit, error_instruction:str) -> stim.Circuit:
    circuit_str = str(circuit)    
    ## Find all the unique cx instructions
    cx_instructions = re.findall('CX.*\n', circuit_str)
    unique_cx_instructions = list(set(cx_instructions))
    unique_cx_instructions
    
    ## Add gate errors after each cx instruction
    for cx_ins in unique_cx_instructions:
        circuit_str = circuit_str.replace(cx_ins, cx_ins + 
                                      cx_ins.replace('CX', error_instruction))
    
    modified_circuit = stim.Circuit(circuit_str)
    

    return modified_circuit


In [27]:
import multiprocessing as mp
import numpy as np
import time
import bposd
from ldpc.codes import ring_code
from bposd.hgp import hgp
from bposd import bposd_decoder
from ldpc.bplsd_decoder import BpLsdDecoder
import multiprocessing as mp
from bposd.css import css_code
import sys
sys.path.append('./src/')
from scipy import sparse
from ldpc.codes import rep_code
from pymatching import Matching
import secrets

def hypergraph_product_code1(H1, H2):
    # 获取矩阵的形状
    m1, n1 = H1.shape
    m2, n2 = H2.shape

    # 构造单位矩阵
    I1 = np.eye(n1, dtype=int)
    I2 = np.eye(n2, dtype=int)
    I3 = np.eye(m1, dtype=int)
    I4 = np.eye(m2, dtype=int)
    vectors = np.zeros(n2,dtype=int)
    vectors[0]=1
    # 构造 H_X
    HZ = np.hstack((np.kron(I2,H1)%2, np.kron(H2.T,I3)%2))
    LX = np.hstack((np.kron(vectors,np.ones(n1,dtype=int))%2, np.zeros(m2*m1,dtype=int)%2))
    LX = np.atleast_2d(LX)    
    # 构造 H_Z
    HX = np.hstack((np.kron(H2,I1)%2, np.kron(I4,H1.T)%2))
    
    return HX, HZ,LX
# 假设的LFR计算函数
def LFR(logical_vals):
    failures = [1 * logical.any() for logical in logical_vals]
    LFP = np.sum(failures) / 100
    return LFP
Lall = [4]

def generate_channel_matrix(m, length, channel_probs, seed):
    np.random.seed(seed)
    binary_matrix = np.zeros((m, length), dtype=np.uint8)
    chunk_size = 1000
    for i in range(0, m, chunk_size):
        end = min(i + chunk_size, m)
        rand_chunk = np.random.rand(end - i, length)
        binary_matrix[i:end] = (rand_chunk < channel_probs).astype(np.uint8)
    return binary_matrix
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX": 1, "p_idling_gate": 1,"p_H": 1,"p_id": 1}
eval_logical_type = 'Z'
# circuit_type = 'colorproduct'
circuit_type = 'coloration'
def process_sample_batch(num_qubits,eval_p,h,L,num_rep,channel_probs, circuit_error_params, num_samples, start_index, batch_size,circuit_type,D=1,seed=None):
    """
    处理一批采样任务，每个批次执行 `batch_size` 次采样。
    """

    #matching_decoding= Matching_Decoding()
    #matching_decoding.from_detector_error_model(dem, num_logicals)
    #logical_correction = matching_decoding.decode_batch1(detector_vals)
    #corrected_logical_vals = (logical_vals + logical_correction) % 2
    seed= secrets.randbelow(2**32)
    decoder_params = {'max_iter':int(1000), 'bp_method': 'min_sum', 
                      'ms_scaling_factor': 0.9, 'osd_method': "osd_cs", 'osd_order':5}
    max_iter = decoder_params['max_iter']
    bp_method =decoder_params['bp_method']
    ms_scaling_factor = decoder_params['ms_scaling_factor']
    osd_method = decoder_params['osd_method']
    osd_order = decoder_params['osd_order']
    bposd_decoding =BpOsdDecoder(
                    h,
                    channel_probs=channel_probs,
                    max_iter=max_iter,
                    bp_method=bp_method,
                    ms_scaling_factor=ms_scaling_factor,
                    osd_method=osd_method,
                    osd_order=osd_order, )  


    sample = generate_channel_matrix(batch_size,h.shape[1], channel_probs,seed)
    case3_count =0
    case2_count =0 
    case1_count =0
    case4_count =0
    case5_count =0
    LFR_case =[] 
    for i in range(sample.shape[0]):

       vector = sample[i,:]
       vector = np.array(vector)
       syn = h@vector.T%2


       
       error_estimate = bposd_decoding.decode(syn.T)
       corrected_vector = (vector + error_estimate) % 2
       predicted_observables = (corrected_vector @ L.T) % 2
       obs = predicted_observables
       if np.all(obs[:-2] == 0):### 没有任何错误
            case1_count += 1
       if obs[0]== 1 :
            case3_count += 1
       if obs[1]== 1 :
            case4_count += 1
       if (obs[1]== 1 ) and (obs[0]== 1 ):
           case5_count += 1
    # 计算三种情况的比率（分母根据实际样本量调整）
    total = sample.shape[0]
    LFR_case1 = case1_count / total
    LFR_case2 = case2_count / total
    LFR_case3 = case3_count / total
    LFR_case4 = case4_count / total
    LFR_case5 = case5_count / total
    LFR_case.append(1-LFR_case1)
    LFR_case.append(1-LFR_case2)
    LFR_case.append(LFR_case3)
    LFR_case.append(LFR_case4)
    LFR_case.append(LFR_case5)
    del sample
    import gc
    gc.collect()
    return LFR_case

    #

def parallel_sampling(num_qubit,eval_p, H,L,num_rep,channel_prob, circuit_error_params, circuit_type, D, num_samples, batch_size, num_processes=None):
    """
    对单个 `eval_p` 进行 8000 次采样，并行化每次采样任务。
    """
    num_samples = 1000
    batch_size = 20
    num_batches = num_samples // batch_size

    with mp.Pool(50) as pool:
        results = pool.starmap(process_sample_batch, 
                              [(num_qubit,eval_p, H,L,num_rep,channel_prob, circuit_error_params,num_samples , i * batch_size, batch_size,circuit_type, D) 
                               for i in range(num_batches)])
    
    all_case1 = [r[0] for r in results]
    all_case2 = [r[1] for r in results]
    all_case3 = [r[2] for r in results]
    all_case4 = [r[3] for r in results]
    all_case5 = [r[4] for r in results]
    
    # 合并结果
    LFR_case1 = np.mean(all_case1)
    LFR_case2 = np.mean(all_case2)
    LFR_case3 = np.mean(all_case3)
    LFR_case4 = np.mean(all_case4)
    LFR_case5 = np.mean(all_case5)
    return LFR_case1, LFR_case2, LFR_case3,LFR_case4,LFR_case5
eval_ps = np.array([0.0010,0.0020])
#eval_ps = np.array([0.009])
##############################################

ell, m = 15, 3
code = [9, 1, 2, 0, 2, 7] 
H_X, H_Z = construct_ldpc(ell, m, code)

H_X = H_X.astype(np.uint8) 
H_Z = H_Z.astype(np.uint8)

H_X1= gf2_rref(H_X)
H_X1 = remove_zero_rows(H_X1)

H_Z1= gf2_rref(H_Z)
H_Z1 = remove_zero_rows(H_Z1)
LX,LZ,I_s = CSS_code_Logical(H_X1,H_Z1)
LX = LX.astype(np.uint8)

L =10
J_X_1 = LX[-2:,]

h = rep_code(2)
h1 = h.toarray()
D=10
num_rep = 30
surface_codehx,surface_codehz,surface_codelx =hypergraph_product_code1(H1=h1,H2=h1)   
hx=surface_codehx
hz=surface_codehz
lx=surface_codelx
hx=hx.astype(np.uint8)
hz=hz.astype(np.uint8)
LX1,LZ1,I_s1 = CSS_code_Logical(hx,hz)


num_logicals = LX.shape[0]
num_qubit = LX.shape[1]+2*lx.shape[1]
eval_pm = 0.001 
qec_circuit = QECCircuit_OneStage(H_Z,H_X,J_X_1,LX,LZ,I_s,hz,hx,lx,LZ1,num_rep, circuit_error_params, eval_pm,circuit_type,D)


dem = qec_circuit.detector_error_model(flatten_loops=True)#, allow_gauge_detectors=True)


H, LX, channel_prob = GenDecodingGraphs(str(dem), num_logicals=num_logicals)

H = H.astype(np.uint8)
LX = LX.astype(np.uint8)
print(H.shape)
channel_prob = np.array(channel_prob)
print(channel_prob)
LFPPP1 = []
LFPPP2 = []
LFPPP3 = []
LFPPP4 = []
LFPPP5 = []
start_time = time.time()
for eval_p in eval_ps:
   
    LFR_all_result1 = []
    LFR_all_result2 = []
    LFR_all_result3 = []
    LFR_all_result4 = []
    LFR_all_result5 = []
    for L in Lall:

      channel_probs = (eval_p / eval_pm) * channel_prob
      channel_probs = list(channel_probs)
      LFR_case1, LFR_case2, LFR_case3,LFR_case4,LFR_case5= parallel_sampling(num_qubit,eval_p,H,LX,num_rep,channel_probs,circuit_error_params, circuit_type, D, num_samples=100000, batch_size=2000, num_processes=mp.cpu_count())    
      print(f"LFR1 for eval_p = {eval_p}: {LFR_case1}")
      print(f"LFR2 for eval_p = {eval_p}: {LFR_case2}") 
      
      print(f"LFR3 for eval_p = {eval_p}: {LFR_case3}")  
      print(f"LFR4 for eval_p = {eval_p}: {LFR_case4}")  
      print(f"LFR5 for eval_p = {eval_p}: {LFR_case5}")  
      LFR_all_result1.append(LFR_case1)
      LFR_all_result2.append(LFR_case2)
      LFR_all_result3.append(LFR_case3)
      LFR_all_result4.append(LFR_case4)
      LFR_all_result5.append(LFR_case5)
    LFPPP1.append(LFR_all_result1)
    LFPPP2.append(LFR_all_result2)
    LFPPP3.append(LFR_all_result3)
    LFPPP4.append(LFR_all_result4)
    LFPPP5.append(LFR_all_result5)
print(LFPPP1)
print(LFPPP2)
print(LFPPP3)
print(LFPPP4)
print(LFPPP5)
elapsed_time = time.time() - start_time
print(elapsed_time)
eval_ps = np.array([0.0005,0.0007,0.001,0.0012,0.0015,0.0017,0.0020])
for eval_p in eval_ps:
   
    LFR_all_result1 = []
    LFR_all_result2 = []
    LFR_all_result3 = []
    LFR_all_result4 = []
    LFR_all_result5 = []
    for L in Lall:

      channel_probs = (eval_p / eval_pm) * channel_prob
      channel_probs = list(channel_probs)
      LFR_case1, LFR_case2, LFR_case3,LFR_case4,LFR_case5= parallel_sampling(num_qubit,eval_p,H,LX,num_rep,channel_probs,circuit_error_params, circuit_type, D, num_samples=100000, batch_size=2000, num_processes=mp.cpu_count())    
      print(f"LFR1 for eval_p = {eval_p}: {LFR_case1}")
      print(f"LFR2 for eval_p = {eval_p}: {LFR_case2}") 
      
      print(f"LFR3 for eval_p = {eval_p}: {LFR_case3}")  
      print(f"LFR4 for eval_p = {eval_p}: {LFR_case4}")  
      print(f"LFR5 for eval_p = {eval_p}: {LFR_case5}")  
      LFR_all_result1.append(LFR_case1)
      LFR_all_result2.append(LFR_case2)
      LFR_all_result3.append(LFR_case3)
      LFR_all_result4.append(LFR_case4)
      LFR_all_result5.append(LFR_case5)
    LFPPP1.append(LFR_all_result1)
    LFPPP2.append(LFR_all_result2)
    LFPPP3.append(LFR_all_result3)
    LFPPP4.append(LFR_all_result4)
    LFPPP5.append(LFR_all_result5)
print(LFPPP1)
print(LFPPP2)
print(LFPPP3)
print(LFPPP4)
print(LFPPP5)

我好帥
(45, 90)
wohaoshuai
[]
[]
90
45
0
45
0
{'p_i': 0.0, 'p_state_p': 0.001, 'p_m': 0.001, 'p_CX': 0.001, 'p_idling_gate': 0.001, 'p_H': 0.001, 'p_deg': 0.001}
(1395, 13590)
[0.00053333 0.00053333 0.00544022 ... 0.00292666 0.00292666 0.00292666]
LFR1 for eval_p = 0.001: 0.0010000000000000009
LFR2 for eval_p = 0.001: 1.0
LFR3 for eval_p = 0.001: 0.0
LFR4 for eval_p = 0.001: 0.001
LFR5 for eval_p = 0.001: 0.0
LFR1 for eval_p = 0.002: 0.04700000000000002
LFR2 for eval_p = 0.002: 1.0
LFR3 for eval_p = 0.002: 0.021
LFR4 for eval_p = 0.002: 0.021
LFR5 for eval_p = 0.002: 0.01
[[0.0010000000000000009], [0.04700000000000002]]
[[1.0], [1.0]]
[[0.0], [0.021]]
[[0.001], [0.021]]
[[0.0], [0.01]]
24.591251611709595
LFR1 for eval_p = 0.0005: 0.0
LFR2 for eval_p = 0.0005: 1.0
LFR3 for eval_p = 0.0005: 0.0
LFR4 for eval_p = 0.0005: 0.0
LFR5 for eval_p = 0.0005: 0.0
LFR1 for eval_p = 0.0007: 0.0
LFR2 for eval_p = 0.0007: 1.0
LFR3 for eval_p = 0.0007: 0.0
LFR4 for eval_p = 0.0007: 0.0
LFR5 for eval_p = 0

In [ ]:
#################################################
x = symbols('x')
n =24
k = 6

a_x = 1 + x**2+x**8 + x**15 
b_x = 1 + x**2+x**12+ x**17
A = polynomial_to_circulant(a_x, n)
B = polynomial_to_circulant(b_x, n)

H_X= np.hstack((A,B))

H_X1= gf2_rref(H_X)
H_X1 = remove_zero_rows(H_X1)
H_Z= np.hstack((B.T,A.T))
H_Z1= gf2_rref(H_Z)
H_Z1 = remove_zero_rows(H_Z1)
LX,LZ,I_s = CSS_code_Logical(H_X1,H_Z1)
J_X_1 = LX[:2,]
#####################################
#################
H_Z = block_diagonal(hz,hz)
H_X = block_diagonal(hx,hx)
H_X1= gf2_rref(H_X)
H_X1 = remove_zero_rows(H_X1)
H_Z1= gf2_rref(H_Z)
H_Z1 = remove_zero_rows(H_Z1)
LX,LZ,I_s = CSS_code_Logical(H_X1,H_Z1)

J_X_1 = LX[:2,]

################
#################################################
x = symbols('x')
n =24
k = 6

a_x = 1 + x**2+x**8 + x**15 
b_x = 1 + x**2+x**12+ x**17
A = polynomial_to_circulant(a_x, n)
B = polynomial_to_circulant(b_x, n)

H_X= np.hstack((A,B))

H_X1= gf2_rref(H_X)
H_X1 = remove_zero_rows(H_X1)
H_Z= np.hstack((B.T,A.T))
H_Z1= gf2_rref(H_Z)
H_Z1 = remove_zero_rows(H_Z1)
LX,LZ,I_s = CSS_code_Logical(H_X1,H_Z1)
J_X_1 = LX[:2,]
#####################################